In [ ]:
import openai

client = openai.OpenAI()
messages =[]


In [ ]:
def call_ai():
    response = client.chat.completions.create(model="gpt-4.1-nano", messages=messages)
    message = response.choices[0].message.content
    messages.append({"role" : "assistant" , "content" : message})
    print(f"AI: {message}")    

In [ ]:
while True : 
    message = input("Send a message to the LLM....")
    if message == "quit" or message == "q" :
        break
    else:
        messages.append({
            "role" : "user",
            "content" : message
        })
        print(f"User: {message}")
        call_ai()
        
        
    